In [1]:
import os
import json
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import random
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os
import json
from typing import List, Tuple

# ==================================
# Config
# ==================================
# List of multiple camera roots (add/remove as needed)
FRAMES_ROOTS = [
    "./cam2-resized_frames",
    "./cam3-resized_frames",
]
LABELS_ROOT = "./micro_activity_output"
SEQUENCE_LENGTH = 16

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """
    Extract a numeric frame index from a filename like 'frame_00012.jpg'.
    """
    # Extract frame index safely (in case filenames differ, prevents crashes)
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        # Search backwards for a numeric token
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        # Fallback: keep only digits
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """
    Load per-frame micro activity labels for a given activity.
    """
    # Labels are shared across cameras for the same activity
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []

    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)

    # Sort labels by frame index (keys should be numeric)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        # If keys are not purely numeric, fall back to alphabetical order
        items = sorted(labels_dict.items(), key=lambda x: x[0])

    labels = [lbl for _, lbl in items]
    return labels


def sequence_majority_label(seq_labels: List[str], seq_len: int) -> int:
    """
    Decide the sequence label using majority voting.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    # Count votes for fall and lying
    fall_votes = sum(lbl.startswith(tuple(FALL_KEYWORDS)) for lbl in seq_labels)
    lying_votes = sum(any(k in lbl for k in LYING_KEYWORDS) for lbl in seq_labels)

    if fall_votes >= (seq_len // 2):
        return 1  # Fall
    if lying_votes >= (seq_len // 2):
        return 2  # Lie Down
    return 0  # ADL


def generate_sequences_and_labels(
    frame_roots: List[str],
    labels_root: str,
    seq_len: int = 16
) -> List[Tuple[List[str], int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), build sequences and labels.
    Returns a list of tuples: (list_of_frame_paths, label_id)
    """
    # Iterate over multiple camera roots (no need to merge folders physically)
    data = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))  # for logging

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            # Each subfolder = one video of the same activity from this camera
            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                # Sort frames robustly by numeric index
                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                # Safe alignment: trim to minimum length between frames and labels
                n = min(len(frame_paths), len(labels))
                if n < seq_len:
                    # Skip clips that are too short
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                # Sliding window sequence generation
                num_sequences = (n - seq_len) + 1
                for i in range(num_sequences):
                    seq_paths = use_paths[i:i + seq_len]
                    seq_labels = use_labels[i:i + seq_len]
                    label = sequence_majority_label(seq_labels, seq_len)
                    data.append((seq_paths, label))

    return data


# =======================
# Run
# =======================
if __name__ == "__main__":
    dataset = generate_sequences_and_labels(FRAMES_ROOTS, LABELS_ROOT, SEQUENCE_LENGTH)
    print(f"Total sequences: {len(dataset)}")


[cam2-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam2-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_and_Bed_Rolling
[cam3-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
Total sequences: 702093


In [3]:
# Count the distribution
adl_count = sum(1 for _, label in dataset if label == 0)
fall_count = sum(1 for _, label in dataset if label == 1)
lying_count = sum(1 for _, label in dataset if label == 2)

print(f"Total sequences: {len(dataset)}")
print(f"ADL sequences: {adl_count}")
print(f"Fall sequences: {fall_count}")
print(f"Lie Down sequences: {lying_count}")

# Show example
print("\nExample sequence:")
for i in range(3):  # Show first 3 frames of first sample
    print(dataset[0][0][i])
print("Label (0=ADL,1=Fall,2=Lie Down):", dataset[0][1])

Total sequences: 702093
ADL sequences: 591614
Fall sequences: 43913
Lie Down sequences: 66566

Example sequence:
./cam2-resized_frames/Actor_1_Bed/Actor_1_Bed CAM 2/frame_0.jpg
./cam2-resized_frames/Actor_1_Bed/Actor_1_Bed CAM 2/frame_1.jpg
./cam2-resized_frames/Actor_1_Bed/Actor_1_Bed CAM 2/frame_2.jpg
Label (0=ADL,1=Fall,2=Lie Down): 0


In [4]:
import pickle

# Save the dataset
with open("./f_seq_r_3dcnn-convlstm2d-2cams.pkl", "wb") as f:
    pickle.dump(dataset, f)

print("Dataset saved successfully.")

Dataset saved successfully.


In [2]:
import os
import json
import pickle
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-resized_frames",
    "./cam3-resized_frames",
]
LABELS_ROOT = "./micro_activity_output"
SEQUENCE_LENGTH = 16
STRIDE = 8  

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from a filename like 'frame_00012.jpg'."""
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """Load per-frame micro activity labels for a given activity."""
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []
    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        items = sorted(labels_dict.items(), key=lambda x: x[0])
    labels = [lbl for _, lbl in items]
    return labels


def sequence_majority_label(seq_labels: List[str], seq_len: int) -> int:
    """
    Decide the sequence label using majority voting.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    fall_votes = sum(lbl.startswith(tuple(FALL_KEYWORDS)) for lbl in seq_labels)
    lying_votes = sum(any(k in lbl for k in LYING_KEYWORDS) for lbl in seq_labels)
    if fall_votes >= (seq_len // 2):
        return 1  # Fall
    if lying_votes >= (seq_len // 2):
        return 2  # Lie Down
    return 0  # ADL


def generate_sequences_and_labels(
    frame_roots: List[str],
    labels_root: str,
    seq_len: int = 16,
    stride: int = 8,
) -> List[Tuple[List[str], int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), build sequences and labels.
    Returns a list of tuples: (list_of_frame_paths, label_id)
    """
    assert seq_len > 0, "seq_len must be > 0"
    assert stride > 0, "stride must be > 0"

    data: List[Tuple[List[str], int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                n = min(len(frame_paths), len(labels))
                if n < seq_len:
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                # --- STRIDE-aware sliding window ---
                for i in range(0, n - seq_len + 1, stride):
                    seq_paths = use_paths[i:i + seq_len]
                    seq_labels = use_labels[i:i + seq_len]
                    label = sequence_majority_label(seq_labels, seq_len)
                    data.append((seq_paths, label))

    return data


# =======================
# Run inside Jupyter
# =======================
dataset = generate_sequences_and_labels(
    frame_roots=FRAMES_ROOTS,
    labels_root=LABELS_ROOT,
    seq_len=SEQUENCE_LENGTH,
    stride=STRIDE,
)

print(f"Total sequences: {len(dataset)}")
dist = Counter([y for _, y in dataset])
print("Class distribution (0=ADL, 1=Fall, 2=LieDown):", dict(dist))

# Optional: save pickle
with open(f"f_seq_stride{STRIDE}.pkl", "wb") as f:
    pickle.dump(dataset, f)
print(f"Saved pickle to f_seq_stride{STRIDE}.pkl")


[cam2-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam2-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_and_Bed_Rolling
[cam3-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
Total sequences: 87869
Class distribution (0=ADL, 1=Fall, 2=LieDown): {0: 74068, 2: 8315, 1: 5486}
Saved pickle to f_seq_stride8.pkl


In [1]:
import os
import json
import pickle
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-resized_frames",
    "./cam3-resized_frames",
]
LABELS_ROOT = "./micro_activity_output"
SEQUENCE_LENGTH = 16
STRIDE = 8  # sliding window stride

# Keywords for class detection (binary)
FALL_KEYWORDS = ["fall_"]  


def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from a filename like 'frame_00012.jpg'."""
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """Load per-frame micro activity labels for a given activity."""
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []
    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        items = sorted(labels_dict.items(), key=lambda x: x[0])
    labels = [lbl for _, lbl in items]
    return labels


def sequence_majority_label(seq_labels: List[str], seq_len: int) -> int:
    """
    Decide the sequence label using majority voting.
    Returns: 0 = Non-Fall (ADL + Lie Down + ...), 1 = Fall
    """
    fall_votes = sum(lbl.startswith(tuple(FALL_KEYWORDS)) for lbl in seq_labels)
    if fall_votes >= (seq_len // 2):
        return 1  # Fall
    return 0  # Non-Fall


def generate_sequences_and_labels(
    frame_roots: List[str],
    labels_root: str,
    seq_len: int = 16,
    stride: int = 8,
) -> List[Tuple[List[str], int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), build sequences and labels.
    Returns a list of tuples: (list_of_frame_paths, label_id) with binary labels.
    """
    assert seq_len > 0, "seq_len must be > 0"
    assert stride > 0, "stride must be > 0"

    data: List[Tuple[List[str], int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                n = min(len(frame_paths), len(labels))
                if n < seq_len:
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                # --- STRIDE-aware sliding window ---
                for i in range(0, n - seq_len + 1, stride):
                    seq_paths = use_paths[i:i + seq_len]
                    seq_labels = use_labels[i:i + seq_len]
                    label = sequence_majority_label(seq_labels, seq_len)
                    data.append((seq_paths, label))

    return data


# =======================
# Run inside Jupyter
# =======================
dataset = generate_sequences_and_labels(
    frame_roots=FRAMES_ROOTS,
    labels_root=LABELS_ROOT,
    seq_len=SEQUENCE_LENGTH,
    stride=STRIDE,
)

print(f"Total sequences: {len(dataset)}")
dist = Counter([y for _, y in dataset])
print("Class distribution (0=Non-Fall, 1=Fall):", dict(dist))

# Optional: save pickle
out_path = f"f_seq_stride{STRIDE}_binary.pkl"
with open(out_path, "wb") as f:
    pickle.dump(dataset, f)
print(f"Saved pickle to {out_path}")


[cam2-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam2-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_and_Bed_Rolling
[cam3-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
Total sequences: 87869
Class distribution (0=Non-Fall, 1=Fall): {0: 82383, 1: 5486}
Saved pickle to f_seq_stride8_binary.pkl


In [1]:
import os
import json
import pickle
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-cleaned_frames",
    "./cam3-cleaned_frames",
]
LABELS_ROOT = "./micro_activity_output"
SEQUENCE_LENGTH = 16
STRIDE = 8  

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from a filename like 'frame_00012.jpg'."""
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """Load per-frame micro activity labels for a given activity."""
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []
    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        items = sorted(labels_dict.items(), key=lambda x: x[0])
    labels = [lbl for _, lbl in items]
    return labels


def sequence_majority_label(seq_labels: List[str], seq_len: int) -> int:
    """
    Decide the sequence label using majority voting.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    fall_votes = sum(lbl.startswith(tuple(FALL_KEYWORDS)) for lbl in seq_labels)
    lying_votes = sum(any(k in lbl for k in LYING_KEYWORDS) for lbl in seq_labels)
    if fall_votes >= (seq_len // 2):
        return 1  # Fall
    if lying_votes >= (seq_len // 2):
        return 2  # Lie Down
    return 0  # ADL


def generate_sequences_and_labels(
    frame_roots: List[str],
    labels_root: str,
    seq_len: int = 16,
    stride: int = 8,
) -> List[Tuple[List[str], int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), build sequences and labels.
    Returns a list of tuples: (list_of_frame_paths, label_id)
    """
    assert seq_len > 0, "seq_len must be > 0"
    assert stride > 0, "stride must be > 0"

    data: List[Tuple[List[str], int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                n = min(len(frame_paths), len(labels))
                if n < seq_len:
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                # --- STRIDE-aware sliding window ---
                for i in range(0, n - seq_len + 1, stride):
                    seq_paths = use_paths[i:i + seq_len]
                    seq_labels = use_labels[i:i + seq_len]
                    label = sequence_majority_label(seq_labels, seq_len)
                    data.append((seq_paths, label))

    return data


# =======================
# Run inside Jupyter
# =======================
dataset = generate_sequences_and_labels(
    frame_roots=FRAMES_ROOTS,
    labels_root=LABELS_ROOT,
    seq_len=SEQUENCE_LENGTH,
    stride=STRIDE,
)

print(f"Total sequences: {len(dataset)}")
dist = Counter([y for _, y in dataset])
print("Class distribution (0=ADL, 1=Fall, 2=LieDown):", dict(dist))

# Optional: save pickle
with open(f"f_seq_stride{STRIDE}_cutrepo.pkl", "wb") as f:
    pickle.dump(dataset, f)
print(f"Saved pickle to f_seq_stride{STRIDE}_cutrepo.pkl")


Total sequences: 33144
Class distribution (0=ADL, 1=Fall, 2=LieDown): {0: 27570, 2: 3921, 1: 1653}
Saved pickle to f_seq_stride8_cutrepo.pkl


In [2]:
import os
import json
import pickle
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-cleaned_frames",
    "./cam3-cleaned_frames",
]
LABELS_ROOT = "./micro_activity_output_cleaned"
SEQUENCE_LENGTH = 16
STRIDE = 8  

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from a filename like 'frame_00012.jpg'."""
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """Load per-frame micro activity labels for a given activity."""
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []
    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        items = sorted(labels_dict.items(), key=lambda x: x[0])
    labels = [lbl for _, lbl in items]
    return labels


def sequence_majority_label(seq_labels: List[str], seq_len: int) -> int:
    """
    Decide the sequence label using majority voting.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    fall_votes = sum(lbl.startswith(tuple(FALL_KEYWORDS)) for lbl in seq_labels)
    lying_votes = sum(any(k in lbl for k in LYING_KEYWORDS) for lbl in seq_labels)
    if fall_votes >= (seq_len // 2):
        return 1  # Fall
    if lying_votes >= (seq_len // 2):
        return 2  # Lie Down
    return 0  # ADL


def generate_sequences_and_labels(
    frame_roots: List[str],
    labels_root: str,
    seq_len: int = 16,
    stride: int = 8,
) -> List[Tuple[List[str], int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), build sequences and labels.
    Returns a list of tuples: (list_of_frame_paths, label_id)
    """
    assert seq_len > 0, "seq_len must be > 0"
    assert stride > 0, "stride must be > 0"

    data: List[Tuple[List[str], int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                n = min(len(frame_paths), len(labels))
                if n < seq_len:
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                # --- STRIDE-aware sliding window ---
                for i in range(0, n - seq_len + 1, stride):
                    seq_paths = use_paths[i:i + seq_len]
                    seq_labels = use_labels[i:i + seq_len]
                    label = sequence_majority_label(seq_labels, seq_len)
                    data.append((seq_paths, label))

    return data


# =======================
# Run inside Jupyter
# =======================
dataset = generate_sequences_and_labels(
    frame_roots=FRAMES_ROOTS,
    labels_root=LABELS_ROOT,
    seq_len=SEQUENCE_LENGTH,
    stride=STRIDE,
)

print(f"Total sequences: {len(dataset)}")
dist = Counter([y for _, y in dataset])
print("Class distribution (0=ADL, 1=Fall, 2=LieDown):", dict(dist))

# Optional: save pickle
with open(f"f_seq_stride{STRIDE}_cutrepo.pkl", "wb") as f:
    pickle.dump(dataset, f)
print(f"Saved pickle to f_seq_stride{STRIDE}_cutrepo.pkl")


Total sequences: 33144
Class distribution (0=ADL, 1=Fall, 2=LieDown): {2: 7913, 0: 19811, 1: 5420}
Saved pickle to f_seq_stride8_cutrepo.pkl


In [1]:
import os
import json
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-resized_frames",
    "./cam3-resized_frames",
]
LABELS_ROOT = "./micro_activity_output"

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from a filename like 'frame_00012.jpg'."""
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """Load per-frame micro activity labels for a given activity."""
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []
    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        items = sorted(labels_dict.items(), key=lambda x: x[0])
    labels = [lbl for _, lbl in items]
    return labels


def map_label_to_class(lbl: str) -> int:
    """
    Map a single micro-activity label to class id.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    if any(lbl.startswith(k) for k in FALL_KEYWORDS):
        return 1  # Fall
    if any(k in lbl for k in LYING_KEYWORDS):
        return 2  # Lie Down
    return 0  # ADL


def generate_frame_labels(
    frame_roots: List[str],
    labels_root: str,
) -> List[Tuple[str, int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), and assign a label to EACH FRAME.
    Returns a list of tuples: (frame_path, label_id)
    """
    data: List[Tuple[str, int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                n = min(len(frame_paths), len(labels))
                if n <= 0:
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                for fp, lbl in zip(use_paths, use_labels):
                    cls = map_label_to_class(lbl)
                    data.append((fp, cls))

    return data


# =======================
# Run inside Jupyter
# =======================
per_frame_dataset = generate_frame_labels(
    frame_roots=FRAMES_ROOTS,
    labels_root=LABELS_ROOT,
)

print(f"Total frames (labeled): {len(per_frame_dataset)}")
dist = Counter([y for _, y in per_frame_dataset])


# 0=ADL, 1=Fall, 2=Lie Down
adl_count = dist.get(0, 0)
fall_count = dist.get(1, 0)
liedown_count = dist.get(2, 0)

print("Class distribution (0=ADL, 1=Fall, 2=LieDown):", dict(dist))
print(f"ADL frames: {adl_count}")
print(f"Fall frames: {fall_count}")
print(f"Lie Down frames: {liedown_count}")


[cam2-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam2-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_and_Bed_Rolling
[cam3-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
Total frames (labeled): 704688
Class distribution (0=ADL, 1=Fall, 2=LieDown): {0: 595344, 2: 66111, 1: 43233}
ADL frames: 595344
Fall frames: 43233
Lie Down frames: 66111


In [1]:
import os
import json
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-cleaned_frames",
    "./cam3-cleaned_frames",
]
LABELS_ROOT = "./micro_activity_output_cleaned"

# Keywords for class detection
FALL_KEYWORDS = ["fall_"]
LYING_KEYWORDS = ["lying_", "_lying"]


def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from a filename like 'frame_00012.jpg'."""
    try:
        core = os.path.splitext(filename)[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1


def load_labels_for_activity(labels_root: str, activity_name: str) -> List[str]:
    """Load per-frame micro activity labels for a given activity."""
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return []
    with open(label_json_path, "r") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    try:
        items = sorted(labels_dict.items(), key=lambda x: int(x[0]))
    except Exception:
        items = sorted(labels_dict.items(), key=lambda x: x[0])
    labels = [lbl for _, lbl in items]
    return labels


def map_label_to_class(lbl: str) -> int:
    """
    Map a single micro-activity label to class id.
    Returns: 0=ADL, 1=Fall, 2=Lie Down
    """
    if any(lbl.startswith(k) for k in FALL_KEYWORDS):
        return 1  # Fall
    if any(k in lbl for k in LYING_KEYWORDS):
        return 2  # Lie Down
    return 0  # ADL


def generate_frame_labels(
    frame_roots: List[str],
    labels_root: str,
) -> List[Tuple[str, int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), and assign a label to EACH FRAME.
    Returns a list of tuples: (frame_path, label_id)
    """
    data: List[Tuple[str, int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels = load_labels_for_activity(labels_root, activity_name)
            if not labels:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)
                frame_paths = [os.path.join(frames_dir, f) for f in frame_files]

                n = min(len(frame_paths), len(labels))
                if n <= 0:
                    continue

                use_paths = frame_paths[:n]
                use_labels = labels[:n]

                for fp, lbl in zip(use_paths, use_labels):
                    cls = map_label_to_class(lbl)
                    data.append((fp, cls))

    return data


# =======================
# Run inside Jupyter
# =======================
per_frame_dataset = generate_frame_labels(
    frame_roots=FRAMES_ROOTS,
    labels_root=LABELS_ROOT,
)

print(f"Total frames (labeled): {len(per_frame_dataset)}")
dist = Counter([y for _, y in per_frame_dataset])


# 0=ADL, 1=Fall, 2=Lie Down
adl_count = dist.get(0, 0)
fall_count = dist.get(1, 0)
liedown_count = dist.get(2, 0)

print("Class distribution (0=ADL, 1=Fall, 2=LieDown):", dict(dist))
print(f"ADL frames: {adl_count}")
print(f"Fall frames: {fall_count}")
print(f"Lie Down frames: {liedown_count}")


Total frames (labeled): 267179
Class distribution (0=ADL, 1=Fall, 2=LieDown): {2: 63233, 0: 161017, 1: 42929}
ADL frames: 161017
Fall frames: 42929
Lie Down frames: 63233


In [3]:
# per_frame_labeling_official_map.py
import os
import json
import re
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-cleaned_frames",
    "./cam3-cleaned_frames",
]
LABELS_ROOT = "./micro_activity_output_cleaned"

# ==================================
# Official micro -> macro mapping (same as udfall_microactivity_stats.py)
# Labels are normalized (lowercase, '_' -> ' ', collapse spaces) before lookup.
# ==================================
OFFICIAL_MAP = {
    "fall lateral": "Fall",
    "fall frontal": "Fall",
    "fall crouch": "Fall",
    "fall rolling": "Fall",
    "sit up from lying": "ADL",
    "stand still": "ADL",
    "lie down from sitting": "ADL",
    "sit down from standing": "ADL",
    "stand up from floor": "ADL",
    "rolling bed": "ADL",
    "sit still": "ADL",
    "stand up from sit": "ADL",
    "walking": "ADL",
    "pick up object": "ADL",
    "lie down on the floor": "ADL",
    "lie still": "Lying",
}

def _normalize_label(s: str) -> str:
    """Lowercase, replace underscores with spaces, and collapse multiple spaces."""
    s = s.lower().replace("_", " ").strip()
    return re.sub(r"\s+", " ", s)

def to_macro(label: str) -> str:
    """Map a micro-activity to its macro group using the official table (default 'ADL')."""
    return OFFICIAL_MAP.get(_normalize_label(label), "ADL")

# Map macro name to numeric class id
MACRO_TO_ID = {"ADL": 0, "Fall": 1, "Lying": 2}

# ==================================
# Helpers
# ==================================
def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from names like 'frame_00012.jpg' (returns -1 if not found)."""
    try:
        core = os.path.splitext(os.path.basename(filename))[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1

def load_labels_dict(labels_root: str, activity_name: str) -> dict:
    """
    Load per-frame labels for a given activity as a dict[int -> str].
    Supports either {'root': {...}} or a flat dict.
    """
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return {}
    with open(label_json_path, "r", encoding="utf-8") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    # Coerce keys to int when possible
    out = {}
    for k, v in labels_dict.items():
        try:
            out[int(k)] = v
        except Exception:
            # skip non-numeric keys
            continue
    return out

def map_label_to_class(lbl: str) -> int:
    """Use the official micro→macro mapping; return numeric class id."""
    macro = to_macro(lbl)
    return MACRO_TO_ID[macro]

# ==================================
# Core
# ==================================
def generate_frame_labels_index_aware(
    frame_roots: List[str],
    labels_root: str,
) -> List[Tuple[str, int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), and assign a label to EACH FRAME.
    Index-aware alignment: for each frame, extract its numeric index and look up the label by that index.
    Returns a list of tuples: (frame_path, class_id)
    """
    data: List[Tuple[str, int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels_dict = load_labels_dict(labels_root, activity_name)
            if not labels_dict:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)

                local_count = 0
                skipped_no_label = 0

                for fname in frame_files:
                    idx = safe_frame_index(fname)
                    if idx < 0:
                        continue
                    lbl = labels_dict.get(idx)
                    if lbl is None:
                        skipped_no_label += 1
                        continue  # no label for this frame index
                    cls = map_label_to_class(lbl)
                    data.append((os.path.join(frames_dir, fname), cls))
                    local_count += 1

                if skipped_no_label > 0:
                    print(f"[{cam_tag}] {activity_name}/{os.path.basename(frames_dir)}: "
                          f"labeled={local_count}, skipped_no_label={skipped_no_label}")

    return data

# ==================================
# Run
# ==================================
if __name__ == "__main__":
    per_frame_dataset = generate_frame_labels_index_aware(
        frame_roots=FRAMES_ROOTS,
        labels_root=LABELS_ROOT,
    )

    print(f"Total frames (labeled): {len(per_frame_dataset)}")
    dist = Counter([y for _, y in per_frame_dataset])

    # 0=ADL, 1=Fall, 2=Lie Down
    adl_count = dist.get(0, 0)
    fall_count = dist.get(1, 0)
    liedown_count = dist.get(2, 0)

    print("Class distribution (0=ADL, 1=Fall, 2=Lying):", dict(dist))
    print(f"ADL frames: {adl_count}")
    print(f"Fall frames: {fall_count}")
    print(f"Lying frames: {liedown_count}")


Total frames (labeled): 267179
Class distribution (0=ADL, 1=Fall, 2=Lying): {0: 221380, 1: 42929, 2: 2870}
ADL frames: 221380
Fall frames: 42929
Lying frames: 2870


In [4]:
# per_frame_labeling_official_map.py
import os
import json
import re
from collections import Counter
from typing import List, Tuple

# ==================================
# Config
# ==================================
FRAMES_ROOTS = [
    "./cam2-resized_frames",
    "./cam3-resized_frames",
]
LABELS_ROOT = "./micro_activity_output"

# ==================================
# Official micro -> macro mapping (same as udfall_microactivity_stats.py)
# Labels are normalized (lowercase, '_' -> ' ', collapse spaces) before lookup.
# ==================================
OFFICIAL_MAP = {
    "fall lateral": "Fall",
    "fall frontal": "Fall",
    "fall crouch": "Fall",
    "fall rolling": "Fall",
    "sit up from lying": "ADL",
    "stand still": "ADL",
    "lie down from sitting": "ADL",
    "sit down from standing": "ADL",
    "stand up from floor": "ADL",
    "rolling bed": "ADL",
    "sit still": "ADL",
    "stand up from sit": "ADL",
    "walking": "ADL",
    "pick up object": "ADL",
    "lie down on the floor": "ADL",
    "lie still": "Lying",
}

def _normalize_label(s: str) -> str:
    """Lowercase, replace underscores with spaces, and collapse multiple spaces."""
    s = s.lower().replace("_", " ").strip()
    return re.sub(r"\s+", " ", s)

def to_macro(label: str) -> str:
    """Map a micro-activity to its macro group using the official table (default 'ADL')."""
    return OFFICIAL_MAP.get(_normalize_label(label), "ADL")

# Map macro name to numeric class id
MACRO_TO_ID = {"ADL": 0, "Fall": 1, "Lying": 2}

# ==================================
# Helpers
# ==================================
def safe_frame_index(filename: str) -> int:
    """Extract a numeric frame index from names like 'frame_00012.jpg' (returns -1 if not found)."""
    try:
        core = os.path.splitext(os.path.basename(filename))[0]
        tokens = core.split("_")
        for tok in reversed(tokens):
            if tok.isdigit():
                return int(tok)
        digits = "".join(ch for ch in core if ch.isdigit())
        return int(digits) if digits else -1
    except Exception:
        return -1

def load_labels_dict(labels_root: str, activity_name: str) -> dict:
    """
    Load per-frame labels for a given activity as a dict[int -> str].
    Supports either {'root': {...}} or a flat dict.
    """
    label_json_path = os.path.join(labels_root, activity_name, "ann", "per_frame_micro_activities.json")
    if not os.path.exists(label_json_path):
        return {}
    with open(label_json_path, "r", encoding="utf-8") as f:
        loaded_json = json.load(f)
        labels_dict = loaded_json.get("root", loaded_json)
    # Coerce keys to int when possible
    out = {}
    for k, v in labels_dict.items():
        try:
            out[int(k)] = v
        except Exception:
            # skip non-numeric keys
            continue
    return out

def map_label_to_class(lbl: str) -> int:
    """Use the official micro→macro mapping; return numeric class id."""
    macro = to_macro(lbl)
    return MACRO_TO_ID[macro]

# ==================================
# Core
# ==================================
def generate_frame_labels_index_aware(
    frame_roots: List[str],
    labels_root: str,
) -> List[Tuple[str, int]]:
    """
    Iterate over multiple frame roots (e.g., cam2, cam3), and assign a label to EACH FRAME.
    Index-aware alignment: for each frame, extract its numeric index and look up the label by that index.
    Returns a list of tuples: (frame_path, class_id)
    """
    data: List[Tuple[str, int]] = []

    for frames_root in frame_roots:
        if not os.path.isdir(frames_root):
            print(f"[WARN] Frames root not found: {frames_root}")
            continue

        cam_tag = os.path.basename(frames_root.rstrip("/"))

        for activity_name in sorted(os.listdir(frames_root)):
            activity_path = os.path.join(frames_root, activity_name)
            if not os.path.isdir(activity_path):
                continue

            labels_dict = load_labels_dict(labels_root, activity_name)
            if not labels_dict:
                print(f"[{cam_tag}] Missing or empty labels for activity: {activity_name}")
                continue

            for subfolder in sorted(os.listdir(activity_path)):
                frames_dir = os.path.join(activity_path, subfolder)
                if not os.path.isdir(frames_dir):
                    continue

                frame_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(".jpg")]
                if not frame_files:
                    continue

                frame_files.sort(key=safe_frame_index)

                local_count = 0
                skipped_no_label = 0

                for fname in frame_files:
                    idx = safe_frame_index(fname)
                    if idx < 0:
                        continue
                    lbl = labels_dict.get(idx)
                    if lbl is None:
                        skipped_no_label += 1
                        continue  # no label for this frame index
                    cls = map_label_to_class(lbl)
                    data.append((os.path.join(frames_dir, fname), cls))
                    local_count += 1

                if skipped_no_label > 0:
                    print(f"[{cam_tag}] {activity_name}/{os.path.basename(frames_dir)}: "
                          f"labeled={local_count}, skipped_no_label={skipped_no_label}")

    return data

# ==================================
# Run
# ==================================
if __name__ == "__main__":
    per_frame_dataset = generate_frame_labels_index_aware(
        frame_roots=FRAMES_ROOTS,
        labels_root=LABELS_ROOT,
    )

    print(f"Total frames (labeled): {len(per_frame_dataset)}")
    dist = Counter([y for _, y in per_frame_dataset])

    # 0=ADL, 1=Fall, 2=Lie Down
    adl_count = dist.get(0, 0)
    fall_count = dist.get(1, 0)
    liedown_count = dist.get(2, 0)

    print("Class distribution (0=ADL, 1=Fall, 2=Lying):", dict(dist))
    print(f"ADL frames: {adl_count}")
    print(f"Fall frames: {fall_count}")
    print(f"Lying frames: {liedown_count}")


[cam2-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam2-resized_frames] Actor_6_Walk_Full_PH/Actor_6_Walk_Full_PH CAM 2: labeled=14396, skipped_no_label=9424
[cam2-resized_frames] Actor_7_Bed_Rolling_Full_PH/Actor_7_Bed_Rolling_Full_PH CAM 2: labeled=4858, skipped_no_label=2
[cam2-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_Rolling_Full_PH
[cam3-resized_frames] Missing or empty labels for activity: Actor_5_Bed_and_Bed_Rolling
[cam3-resized_frames] Actor_6_Walk_Full_PH/Actor_6_Walk_Full_PH CAM 3: labeled=14396, skipped_no_label=9424
[cam3-resized_frames] Actor_7_Bed_Rolling_Full_PH/Actor_7_Bed_Rolling_Full_PH CAM 3: labeled=4858, skipped_no_label=2
[cam3-resized_frames] Missing or empty labels for activity: Actor_7_Chair_Full_PH_LegBrace
Total frames (labeled): 704688
Class distribution (0=ADL, 1=Fall, 2=Lying): {0: 658889, 1: 42929, 2: 2